# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Read the 2024 school data from a CSV file into a DataFrame named 'df'.
df = pd.read_csv('AFD_ESCOLAS_2024.csv')
# Read the 2023 school data from another CSV file into a DataFrame named 'df1'.
df1 = pd.read_csv('AFD_ESCOLAS_2023_v1.1.csv')

# Concatenate (combine) the two DataFrames (df1 and df) into a single DataFrame.
tabela_final = pd.concat([df1, df])

# Filter the combined DataFrame to only include rows where the 'Ano' column is 2023 or 2024.
tabela_final = tabela_final[tabela_final['Ano'].isin([2023, 2024])]

# Replace all occurrences of '--' with NaN (Not a Number) to represent missing values.
tabela_final = tabela_final.replace('--', np.nan) #transformando os valores '--' como  NaN

# Create a list containing the names of all columns that should be converted to a numeric type.
transformar_float = ['EI_grupo1', 'EI_grupo2', 'EI_grupo3', 'EI_grupo4', 'EI_grupo5',
       'EF_total_grupo1', 'EF_total_grupo2', 'EF_total_grupo3',
       'EF_total_grupo4', 'EF_total_grupo5', 'EF_iniciais_grupo1',
       'EF_iniciais_grupo2', 'EF_iniciais_grupo3', 'EF_iniciais_grupo4',
       'EF_iniciais_grupo5', 'EF_finais_grupo1', 'EF_finais_grupo2',
       'EF_finais_grupo3', 'EF_finais_grupo4', 'EF_finais_grupo5', 'EM_grupo1',
       'EM_grupo2', 'EM_grupo3', 'EM_grupo4', 'EM_grupo5', 'EJA_EF_grupo1',
       'EJA_EF_grupo2', 'EJA_EF_grupo3', 'EJA_EF_grupo4', 'EJA_EF_grupo5',
       'EJA_EM_grupo1', 'EJA_EM_grupo2', 'EJA_EM_grupo3', 'EJA_EM_grupo4',
       'EJA_EM_grupo5'] #definindo uma variável para todas as colunas

# For the specified columns, replace the comma decimal separator with a period and convert the type to float.
tabela_final[transformar_float] = tabela_final[transformar_float].replace(',', '.', regex=True).astype(float)
# Convert the same list of columns to the float data type again.
tabela_final[transformar_float] = tabela_final[transformar_float].astype(float) #transformando as colunas dos grupos dos docentes em float

# Save the processed DataFrame to a new CSV file.
# index=False prevents writing the DataFrame index.
# encoding="latin-1" specifies the character encoding.
# sep=';' sets the semicolon as the column separator.
# decimal=',' sets the comma as the decimal separator for numbers.
tabela_final.to_csv('INEP_adequacao_formacao_docente.csv',index=False, encoding="latin-1", sep=';', decimal=',')

# Rename columns to a standardized snake_case format for easier access.
tabela_final = tabela_final.rename(columns={"Ano": "ano", "Região": "nome_regiao", "UF": "sigla_uf", "Código do Município":"id_municipio", "Nome do Município":"nome_municipio", "Código da Escola":"id_escola", "Nome da Escola":"nome_escola", "Localização":"localizacao", "Dependência Administrativa":"dependencia_adm", "EI_grupo1":"ei_grupo1", "EI_grupo2":"ei_grupo2", "EI_grupo3":"ei_grupo3", "EI_grupo4":"ei_grupo4", "EI_grupo5":"ei_grupo5", "EF_total_grupo1":"ef_total_grupo1", "EF_total_grupo2":"ef_total_grupo2", "EF_total_grupo3":"ef_total_grupo3", "EF_total_grupo4":"ef_total_grupo4", "EF_total_grupo5":"ef_total_grupo5", "EF_iniciais_grupo1":"ef_iniciais_grupo1", "EF_iniciais_grupo2":"ef_iniciais_grupo2", "EF_iniciais_grupo3":"ef_iniciais_grupo3", "EF_iniciais_grupo4":"ef_iniciais_grupo4", "EF_iniciais_grupo5":"ef_iniciais_grupo5", "EF_finais_grupo1":"ef_finais_grupo1", "EF_finais_grupo2":"ef_finais_grupo2", "EF_finais_grupo3":"ef_finais_grupo3", "EF_finais_grupo4":"ef_finais_grupo4", "EF_finais_grupo5":"ef_finais_grupo5", "EM_grupo1":"em_grupo1", "EM_grupo2":"em_grupo2", "EM_grupo3":"em_grupo3", "EM_grupo4":"em_grupo4", "EM_grupo5":"em_grupo5", "EJA_EF_grupo1":"eja_ef_grupo1", "EJA_EF_grupo2":"eja_ef_grupo2", "EJA_EF_grupo3":"eja_ef_grupo3", "EJA_EF_grupo4":"eja_ef_grupo4", "EJA_EF_grupo5":"eja_ef_grupo5", "EJA_EM_grupo1":"eja_em_grupo1", "EJA_EM_grupo2":"eja_em_grupo2", "EJA_EM_grupo3":"eja_em_grupo3", "EJA_EM_grupo4":"eja_em_grupo4","EJA_EM_grupo5":"eja_em_grupo5"})

# Perform a second renaming operation on some columns, replacing underscores with periods.
tabela_final = tabela_final.rename(columns= {'ef_total_grupo1': 'ef.total_grupo1',
                        'ef_total_grupo2': 'ef.total_grupo2',
                        'ef_total_grupo3': 'ef.total_grupo3',
                        'ef_total_grupo4': 'ef.total_grupo4',
                        'ef_total_grupo5': 'ef.total_grupo5',
                        'ef_finais_grupo1' : 'ef.finais_grupo1',
                        'ef_finais_grupo2' : 'ef.finais_grupo2',
                        'ef_finais_grupo3' : 'ef.finais_grupo3',
                        'ef_finais_grupo4' : 'ef.finais_grupo4',
                        'ef_finais_grupo5' : 'ef.finais_grupo5',
                        'eja_ef_grupo1' : 'eja.ef_grupo1',
                        'eja_ef_grupo2' : 'eja.ef_grupo2',
                        'eja_ef_grupo3' : 'eja.ef_grupo3',
                        'eja_ef_grupo4' : 'eja.ef_grupo4',
                        'eja_ef_grupo5' : 'eja.ef_grupo5',
                        'eja_em_grupo1' : 'eja.em_grupo1',
                        'eja_em_grupo2' : 'eja.em_grupo2',
                        'eja_em_grupo3' : 'eja.em_grupo3',
                        'eja_em_grupo4' : 'eja.em_grupo4',
                        'eja_em_grupo5' : 'eja.em_grupo5',
                        'ef_iniciais_grupo1' : 'ef.iniciais_grupo1',
                        'ef_iniciais_grupo2' : 'ef.iniciais_grupo2',
                        'ef_iniciais_grupo3' : 'ef.iniciais_grupo3',
                        'ef_iniciais_grupo4' : 'ef.iniciais_grupo4',
                        'ef_iniciais_grupo5' : 'ef.iniciais_grupo5' })

# Unpivot the DataFrame from a wide format to a long format.
# id_vars are the columns to keep as identifiers.
# value_vars are the columns to unpivot.
# var_name is the name for the new column holding the original column names.
# value_name is the name for the new column holding the values.
tabela_final = tabela_final.melt(id_vars =['ano', 'nome_regiao', 'sigla_uf', 'id_municipio','nome_municipio','id_escola','nome_escola','localizacao','dependencia_adm'],
        value_vars =['ei_grupo1', 'ei_grupo2', 'ei_grupo3', 'ei_grupo4', 'ei_grupo5',
       'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.iniciais_grupo1',
       'ef.iniciais_grupo2', 'ef.iniciais_grupo3', 'ef.iniciais_grupo4',
       'ef.iniciais_grupo5', 'ef.finais_grupo1', 'ef.finais_grupo2',
       'ef.finais_grupo3', 'ef.finais_grupo4', 'ef.finais_grupo5', 'em_grupo1',
       'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'eja.ef_grupo1',
       'eja.ef_grupo2', 'eja.ef_grupo3', 'eja.ef_grupo4', 'eja.ef_grupo5',
       'eja.em_grupo1', 'eja.em_grupo2', 'eja.em_grupo3', 'eja.em_grupo4',
       'eja.em_grupo5'],
        var_name ='grupos_docentes_por_ano_escolar',
        value_name ='per_valor')

# Create a new DataFrame 'df1' by splitting the 'grupos_docentes_por_ano_escolar' column from the original 'df' DataFrame.
df1= df['grupos_docentes_por_ano_escolar'].str.split('_', n=1, expand=True)
# Rename the columns of the new 'df1' DataFrame.
df1.columns = ['ano_escolar', 'docente_grupos']

# Define a function to replace abbreviations in a string with their full descriptions using regular expressions.
def transformar (nome):
    nome = re.sub(r"\bei\b","Ensino Infantil", nome)
    nome = re.sub(r"\bef.total\b","Ensino Fundamental - Todos os anos", nome)
    nome = re.sub(r"\bef.iniciais\b","Ensino Fundamental - Anos Iniciais", nome)
    nome = re.sub(r"\bef.finais\b","Ensino Fundamental - Anos Finais", nome)
    nome = re.sub(r"\beja.em\b","Ensino Médio do EJA", nome)
    nome = re.sub(r"\beja.ef\b","Ensino Fundamental do EJA", nome)
    nome = re.sub(r"\bgrupo1\b","Grupo 1", nome)
    nome = re.sub(r"\bgrupo2\b","Grupo 2", nome)
    nome = re.sub(r"\bgrupo3\b","Grupo 3", nome)
    nome = re.sub(r"\bgrupo4\b","Grupo 4", nome)
    nome = re.sub(r"\bgrupo5\b","Grupo 5", nome)
    return nome

# Apply the 'transformar' function to the 'ano_escolar' column of df1.
df1['ano_escolar'] = df1['ano_escolar'].apply(transformar)
# Apply the 'transformar' function to the 'docente_grupos' column of df1.
df1['docente_grupos'] = df1['docente_grupos'].apply(transformar)

# In the 'ano_escolar' column, replace the specific value 'em' with 'Ensino Médio'.
df1['ano_escolar'] = df1['ano_escolar'].replace('em', 'Ensino Médio')

# Concatenate the original 'df' DataFrame with the new 'df1' DataFrame along the columns (side-by-side).
df = pd.concat([df, df1], axis=1)

# Drop the 'grupos_docentes_por_ano_escolar' column from the concatenated DataFrame.
df = df.drop(columns='grupos_docentes_por_ano_escolar')

# Select a specific subset and order of columns for the DataFrame 'df'.
df = df[['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm','ano_escolar', 'docente_grupos', 'per_valor']]

# Convert the 'per_valor' column to string, replace commas with periods, and then convert to float.
df['per_valor'] = df['per_valor'].astype(str).str.replace(',', '.').astype(float)

# Convert data types for multiple columns using a dictionary for precision.
# Converter tipos de dados para igualar à segunda tabela
df = df.astype({
    'ano': 'int64',
    'id_municipio': 'int64',
    'id_escola': 'int64',
    'nome_regiao': 'object',
    'sigla_uf': 'object',
    'nome_municipio': 'object',
    'nome_escola': 'object',
    'localizacao': 'object',
    'dependencia_adm': 'object',
    'ano_escolar': 'object',
    'docente_grupos': 'object',
    'per_valor': 'float64'
})

# Select a final subset of columns from the DataFrame 'df'.
df = df[['ano', 'sigla_uf', 'ano_escolar', 'dependencia_adm', 'localizacao', 'per_valor']]

# Upload

In [ ]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação referente a observação'),
          bigquery.SchemaField('localizacao', 'STRING', description= 'Se a área da observação é urbana ou rural'),
          bigquery.SchemaField('dependencia_adm', 'STRING', description= 'Dependência admnistrativa da observação (privada ou pública)'),
          bigquery.SchemaField('ano_escolar', 'STRING', description= 'Ano escolar referente ao qual docente leciona'),
          bigquery.SchemaField('per_valor', 'FLOAT', description= 'Percentual de docências na respectiva unidade da agregação classificadas em cada uma das categorias'),
]

## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('Datalake')

table_ref = dataset_ref.table('INEP_esforco_docente_temporal_v2') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()